In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import linear_model
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

1. Hava durumu modeli

In [2]:
weather_df = pd.read_csv("https://djl-lms-assets.s3.eu-central-1.amazonaws.com/datasets/weatherHistory.csv")
weather_df.head()

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472,7.389,0.890,14.120,251.000,15.826,0.000,1015.130,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.356,7.228,0.860,14.265,259.000,15.826,0.000,1015.630,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.378,9.378,0.890,3.928,204.000,14.957,0.000,1015.940,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.289,5.944,0.830,14.104,269.000,15.826,0.000,1016.410,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.756,6.978,0.830,11.045,259.000,15.826,0.000,1016.510,Partly cloudy throughout the day.


Önceki derste olduğu gibi, hedef değişkeninizin görünür sıcaklık ve sıcaklık arasındaki fark olduğu bir doğrusal regresyon modeli oluşturun. Açıklayıcı değişkenler olarak nem ve rüzgar hızı kullanın. Şimdi, modelinizi OLS kullanarak tahmin edin. R-kare ve ayarlanmış R-kare değerleri nelerdir? Tatminkar olduklarını düşünüyor musunuz? Açıklayın.

In [3]:
weather_df['TGap (C)']=weather_df['Temperature (C)']-weather_df['Apparent Temperature (C)']
Y = weather_df['TGap (C)']
X = weather_df[['Humidity','Wind Speed (km/h)']]
X = sm.add_constant(X)

In [4]:
sonuclar = sm.OLS(Y, X).fit()
sonuclar.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               TGap (C)   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Thu, 27 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:49:47   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -2.4381      0.021   -115.948      0.000      -2.479      -2.397
Humidity              3.0292      0.024    126.479      0.000       2.982       3.076
Wind Speed (km/h)     0.1193      0.001    176.164      0.000       0.118       0.121
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.264
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                           0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

 F istatistiğini okumak yerine, ilgili p-değerini okumak daha kolaydır. P-değeri ne kadar düşükse, modelimiz için o kadar iyidir. Yani, bizim modelimiz için F-testinin p-değeri 0,1'den küçük ya da eşitse (ya da 0.05'e eşit ya da daha az ise), modelimizin yararlı olduğunu ve hedefi açıklamada istatistiksel olarak anlamlı bir şey olduğunu söyleriz. <br>
 P-değeri modelde 0'a yakındır.

 R-kare ve ayarlanmış R-kare (R-kare varyantı) değerleri, modelin hedefteki varyansı ne kadar iyi hesapladığını anlatır.R-squared değeri 0.288 ile çok düşüktür.İyi açıklayabilmesi için 5 ile 7 arasında olması tercih edilir.

Daha sonra, yukarıdaki modele nem ve rüzgar hızı etkileşimini dahil edin ve OLS'yi kullanarak modeli tahmin edin. Şimdi, bu modelin R-kare değeri nedir? Bu model bir öncekine göre gelişti mi?

In [5]:
Y = weather_df['TGap (C)']
weather_df['HumidityWind'] = weather_df['Humidity'] * weather_df['Wind Speed (km/h)']
X = weather_df[['Humidity','Wind Speed (km/h)','HumidityWind']]

X = sm.add_constant(X)

sonuclar = sm.OLS(Y, X).fit()
sonuclar.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               TGap (C)   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Thu, 27 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:57:50   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.0839      0.033     -2.511      0.012      -0.149      -0.018
Humidity             -0.1775      0.043     -4.133      0.000      -0.262      -0.093
Wind Speed (km/h)    -0.0905      0.002    -36.797      0.000      -0.095      -0.086
HumidityWind          0.2971      0.003     88.470      0.000       0.291       0.304
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.262
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                           0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Etkileşimi de modele dahil edince r kare değerinde bir miktar artma olmuştur. Modelin daha iyi bir açıklama yapabilmesi için uygun değişkenler ekleyebiliriz.

İlk modele ek açıklayıcı değişken olarak görünürlük ekleyin ve tahmin edin. R-kare arttı mı? Ayarlanmış R-kare değeri ne oldu? Tabloda ortaya çıkan farklılıkları, ayarlanmış R-kare içindeki iyileşme açısından etkileşim terimi ve görünürlük ile karşılaştırın. Hangisi daha kullanışlı?

In [6]:
Y = weather_df['TGap (C)']
weather_df['HumidityWind'] = weather_df['Humidity'] * weather_df['Wind Speed (km/h)']
X = weather_df[['Humidity','Wind Speed (km/h)','HumidityWind','Visibility (km)']]

X = sm.add_constant(X)

sonuclar = sm.OLS(Y, X).fit()
sonuclar.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               TGap (C)   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.363
Method:                 Least Squares   F-statistic:                 1.377e+04
Date:                Thu, 27 Jan 2022   Prob (F-statistic):               0.00
Time:                        16:01:58   Log-Likelihood:            -1.6504e+05
No. Observations:               96453   AIC:                         3.301e+05
Df Residuals:                   96448   BIC:                         3.301e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.1006      0.039     28.459      0.000       1.025       1.176
Humidity             -0.8909      0.044    -20.263      0.000      -0.977      -0.805
Wind Speed (km/h)    -0.1033      0.002    -42.579      0.000      -0.108      -0.099
HumidityWind          0.3164      0.003     95.355      0.000       0.310       0.323
Visibility (km)      -0.0646      0.001    -58.051      0.000      -0.067      -0.062
==============================================================================
Omnibus:                     5328.364   Durbin-Watson:                   0.285
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11525.074
Skew:                           0.373   Prob(JB):                         0.00
Kurtosis:                       4.520   Cond. No.                         246.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

R-kare'nin, açıklayıcı değişkenlerin sayısı ile artma eğiliminde olduğu için yanlı olduğunu zaten gördük. Bu yüzden, R-kare yerine, ayarlanmış R-kare kullanabiliriz. Daha yüksek ayarlanmış R-kare, hedef değişkeni daha iyi açıklayan model anlamına gelir. (0,341'den 0,363e artış)

AIC ve BIC puanlarına göre yukarıdaki üç modelden en iyisini seçin. Mentor ile gerekçenizi tartışarak seçiminizi doğrulayın.

Bilgi kriterlerini kullanmak da farklı modelleri karşılaştırmak ve en iyisini seçmek için yaygın bir yoldur. Burada Akaike Information Criterion(AIC) ve Bayesian Information Criterion (BIC) olarak bilinen iki bilgi kriterinden bahsediyoruz. Her ikisi de kare hataları toplamını(SSE), örneklem boyutunu ve parametre sayısını dikkate alır.Hem AIC hem de BIC için, daha düşük değer daha iyidir. Az bir fark ile etkileşimli ve görünürlük parametreleri dahil olan model daha iyi AIC ve BIC değerlerine sahiptir.

2. Ev fiyatları modeli

In [8]:
house_prices_df = pd.read_csv("https://djl-lms-assets.s3.eu-central-1.amazonaws.com/datasets/house_prices.csv", sep = ";")
house_prices_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.000,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.000,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.000,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.000,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.000,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.000,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.000,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.000,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.000,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
to_drop = ['Id','Alley','PoolQC','Fence','MiscFeature','Electrical']

for k in to_drop:
    house_prices_df.drop([k], axis = 1, inplace = True)


cat_col_data = ['FireplaceQu','GarageType','GarageFinish','MasVnrType','BsmtQual',
           'BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu',
          'GarageQual','GarageCond']

ncat_col_data = ['LotFrontage','GarageYrBlt','MasVnrArea']
for i in cat_col_data:
    house_prices_df[i] = house_prices_df[i].fillna(house_prices_df[i].mode()[0])
    
for j in ncat_col_data:
    house_prices_df[j] = house_prices_df[j].fillna(house_prices_df[j].mean())
    categorical_features=[feature for feature in house_prices_df.columns if house_prices_df[feature].dtype=='O']
for feature in categorical_features:
    labels_ordered=house_prices_df.groupby([feature])['SalePrice'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    house_prices_df[feature]=house_prices_df[feature].map(labels_ordered)
X = house_prices_df.drop(['SalePrice'], axis = 1)
Y = house_prices_df['SalePrice']
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

Ev fiyatları modelinizi tekrar çalıştırın ve F-testi, R-kare, ayarlanmış R-kare, AIC ve BIC kullanarak modelinizin uygunluğunu değerlendirin.

Adj. R-squared:	0.847 ile overfitting'e işaret ediyor olabilir. F-testi için p parametresine baktığımızda bazı değişkenlerin 0,5 değerinden büyük olduğunu görebiliriz.Bu değişkenleri modelden çıkarmak faydalı olacaktır. Böyle daha düşük AIC ve BIC değerleri de elde etmiş oluruz.

Modelinizin uygunluğunu iyileştirmek için, bazı değişkenleri ekleyerek veya kaldırarak farklı model özelliklerini deneyin.

In [12]:
house_prices_df['TotalSF']=house_prices_df.TotalBsmtSF + house_prices_df.GrLivArea
house_prices_df['TotalFloorSF']=house_prices_df['1stFlrSF']+ house_prices_df['2ndFlrSF']
house_prices_df['TotalPorchSF']=house_prices_df.OpenPorchSF + house_prices_df.EnclosedPorch + house_prices_df['3SsnPorch'] + house_prices_df['ScreenPorch']
house_prices_df['TotalBathroom'] = house_prices_df.FullBath + .5 * house_prices_df.HalfBath + house_prices_df.BsmtFullBath + .5 * house_prices_df.BsmtHalfBath

In [16]:
X = house_prices_df[['OverallQual','TotalFloorSF','YearBuilt','GarageCars','TotalBathroom']]
Y = house_prices_df['SalePrice']
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.754
Method:                 Least Squares   F-statistic:                     896.7
Date:                Thu, 27 Jan 2022   Prob (F-statistic):               0.00
Time:                        16:57:11   Log-Likelihood:                -17517.
No. Observations:                1460   AIC:                         3.505e+04
Df Residuals:                    1454   BIC:                         3.508e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -5.608e+05   9.39e+04     -5.973      0.000   -7.45e+05   -3.77e+05
OverallQual    2.307e+04   1160.477     19.879      0.000    2.08e+04    2.53e+04
TotalFloorSF     49.6922      2.997     16.583      0.000      43.814      55.570
YearBuilt       241.7608     49.480      4.886      0.000     144.701     338.820
GarageCars     1.545e+04   1871.547      8.258      0.000    1.18e+04    1.91e+04
TotalBathroom  1.002e+04   1909.081      5.249      0.000    6275.054    1.38e+04
==============================================================================
Omnibus:                      461.884   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            13217.392
Skew:                           0.855   Prob(JB):                         0.00
Kurtosis:                      17.641   Cond. No.                     2.28e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.28e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
X = house_prices_df[['OverallQual','GrLivArea','Neighborhood','ExterQual']]
Y = house_prices_df['SalePrice']
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.771
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     1227.
Date:                Thu, 27 Jan 2022   Prob (F-statistic):               0.00
Time:                        16:52:16   Log-Likelihood:                -17467.
No. Observations:                1460   AIC:                         3.494e+04
Df Residuals:                    1455   BIC:                         3.497e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -7.306e+04   4802.009    -15.215      0.000   -8.25e+04   -6.36e+04
OverallQual   1.723e+04   1248.713     13.802      0.000    1.48e+04    1.97e+04
GrLivArea       54.7390      2.355     23.244      0.000      50.119      59.358
Neighborhood  3160.5682    217.435     14.536      0.000    2734.049    3587.087
ExterQual     2.011e+04   2634.195      7.632      0.000    1.49e+04    2.53e+04
==============================================================================
Omnibus:                      484.239   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10348.009
Skew:                           1.017   Prob(JB):                         0.00
Kurtosis:                      15.883   Cond. No.                     7.92e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.92e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Adj. R-squared: 0.771 ile ideal model oalrak seçebilirim. AIC ve BIC değerlerinde de diğer modellere göre en düşük modeli elde ettim